In [9]:
import os

In [10]:
%pwd
os.chdir("/Users/rahuln/S-6/Basic_NLP/Text-Summarizer")

In [11]:
# Ensure working directory is project root for config file access
import os
os.chdir("/Users/rahuln/S-6/Basic_NLP/Text-Summarizer")

In [12]:
%pwd

'/Users/rahuln/S-6/Basic_NLP/Text-Summarizer'

In [13]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [14]:
from textsummarizer.constants import *
from textsummarizer.utils.common import read_yaml, create_directories

In [15]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Use list instead of list[Path] for ensure compatibility
        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [16]:
import os
import urllib.request as request
import zipfile
from textsummarizer.logging import logger
from textsummarizer.utils.common import get_size

In [17]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [18]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    print(f"Error: {e}")

[2025-08-11 03:01:57,852:INFO:common: yaml file: config/config.yaml loaded successfully]
[2025-08-11 03:01:57,853:INFO:common: yaml file: params.yaml loaded successfully]
[2025-08-11 03:01:57,853:INFO:common: created directory at: artifacts]
[2025-08-11 03:01:57,854:INFO:common: created directory at: artifacts/data_ingestion]
[2025-08-11 03:01:57,853:INFO:common: yaml file: params.yaml loaded successfully]
[2025-08-11 03:01:57,853:INFO:common: created directory at: artifacts]
[2025-08-11 03:01:57,854:INFO:common: created directory at: artifacts/data_ingestion]
[2025-08-11 03:02:05,646:INFO:1434958058: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosni

In [19]:
config = ConfigurationManager()
print("artifacts_root:", config.config.artifacts_root)

[2025-08-11 03:02:05,773:INFO:common: yaml file: config/config.yaml loaded successfully]
[2025-08-11 03:02:05,775:INFO:common: yaml file: params.yaml loaded successfully]
[2025-08-11 03:02:05,776:INFO:common: created directory at: artifacts]
artifacts_root: artifacts
[2025-08-11 03:02:05,775:INFO:common: yaml file: params.yaml loaded successfully]
[2025-08-11 03:02:05,776:INFO:common: created directory at: artifacts]
artifacts_root: artifacts
